# English Score

Просмотр фильмов на оригинальном языке - это популярный и действенный метод прокачаться при изучении иностранных языков. Важно выбрать фильм, который подходит студенту по уровню сложности, т.е. студент понимал 50-70 % диалогов. Чтобы выполнить это условие, необходимо посмотреть фильм и решить, какому уровню он соответствует. Однако это требует больших временных затрат.
**Задача проекта:** Разработать ML решение для автоматического определения уровня сложности англоязычных фильмов. 

**Цель проекта:**
выбрать лучшую модель для определения уровеня сложности языка фильма по субтитрам.

План работы:

🔸Загрузить датасет.

🔸Сопоставить таргет и признаки, составить общий датасет для работы

🔸В зависимости от выбранных моделей разделить на 2 или 3 выборки

🔸В признаках (текстах) провести подготовку к применению моделей: преобразовать текст в вектор признаков (“мешок слов”, TfidfTransformer или аналоги)

🔸Выбрать модели, попробовать обучить и сравнить результаты разных моделей

🔸Выбрать лучшую модель


<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Сформируем-файл-с-данными-для-обработки" data-toc-modified-id="Сформируем-файл-с- данными-для-обработки-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Сформируем файл с данными для обработки</a></span></li><li><span><a href="#Обработка-данных" data-toc-modified-id="Обработка-данных-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Обработка данных</a></span></li><li><span><a href="#Обработка-субтитров" data-toc-modified-id="Обработка-субтитров-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Обработка субтитров</a></span></li></ul></li><li><span><a href="#Обучение-моделей-для-классификации-документов" data-toc-modified-id="Обучение-моделей-для-классификации-документов-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение моделей для классификации документов</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#LGBMClassifier" data-toc-modified-id="LGBMClassifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LGBMClassifier</a></span></li><li><span><a href="#RandomForestClassifier" data-toc-modified-id="RandomForestClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>RandomForestClassifier</a></span></li><li><span><a href="#GradientBoostingClassifier" data-toc-modified-id="GradientBoostingClassifier-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>GradientBoostingClassifier</a></span></li><li><span><a href="#Сравнение-моделей" data-toc-modified-id="Сравнение-моделей-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Сравнение моделей</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

## Подготовка

### Загрузка данных

In [102]:
!pip install pysrt

In [103]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [104]:
!pip install lightgbm

In [105]:
# Install spaCy (run in terminal/prompt)
import sys
!{sys.executable} -m pip install spacy
# Download spaCy's  'en' Model
!{sys.executable} -m spacy download en
import spacy

     ---------------------------------------- 12.8/12.8 MB 4.9 MB/s eta 0:00:00
[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use
the full pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [106]:
import pandas as pd
import numpy as np
import re
import nltk
import pysrt
import os
import pickle
import lightgbm

from pathlib import Path
from sklearn.datasets import load_files
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight

from catboost import CatBoostClassifier, Pool
from lightgbm import LGBMClassifier


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nata-\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nata-\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Сформируем файл с данными для обработки

Файлы из папки Subtitles_all по уровням

In [107]:
# собрали данные из папок с уровнями
basepath = r'C:\Users\nata-\Documents\Masterskaya\Subtitles_all'

labels = {'A2': 'A2', 'B1': 'B1', 'B2': 'B2', 'C1': 'C1'}
 
names = []
subs = []
targets = []
 
for l in ('A2', 'B1', 'B2', 'C1'):
    path = os.path.join(basepath, l) # собираем все файлы по папкам-уровням
    for file in os.listdir(path): #проходим по списку имен файлов в каталоге
        # открываем файлы из path пока не кончатся файлы
        infile = pysrt.open(os.path.join(path, file), encoding='iso-8859-1')
        base=os.path.basename(file) # возвращает базовое имя пути
        name = os.path.splitext(base)[0] # Разделить путь на имя файла и его расширение
        file_ext = os.path.splitext(base)[1]
        # Если файл не содержит субтитры, пропускаем его
        if file_ext != '.srt':
            continue
 
        txt = infile.text #читаем весь файл
 
        # Сохраняем данные в списки
        names.append(name)
        subs.append(txt)
        targets.append(labels[l])

data = pd.DataFrame({'Movie': names, 'Sub': subs, 'Level': targets}) # собираем датафрейм

In [108]:
data # получили таблицу из всех папок с субтитрами по уровням

,Movie,Sub,Level
0,The Walking Dead-S01E01-Days Gone Bye.English,( bugs chittering )\n( brakes squeak )\n- ( en...,A2
1,The Walking Dead-S01E02-Guts.English,- ( birds chirping )\n- ( bugs chittering )\nB...,A2
2,The Walking Dead-S01E03-Tell It To The Frogs.E...,( thunder rumbling )\nMerle:\nThat's right. Yo...,A2
3,The Walking Dead-S01E04-Vatos.English,( birds chirping )\n- What?\n- Nothing.\nIt's ...,A2
4,The Walking Dead-S01E05-Wildfire.English,"- ( walkie-talkie squawks )\n- Rick: Morgan,\n...",A2
5,The Walking Dead-S01E06-TS-19.English,- ( people yelling )\n- ( radio chatter )\nHey...,A2
6,AmericanBeauty1999.BRRip,"I need a father who's a role model,\nnot some ...",B1
7,Angelas.Christmas.Wish.2020,"Where are we going?\nCome on, Angela.\nBut who...",B1
8,Indiana Jones And The Last Crusade DVDRip Xvid...,"Dismount!\nHerman's horse-sick!\nChaps, no one...",B1
9,mechanic-resurrection_,"Mr. Santos, so good to see you.\nWe saved your...",B1


Откроем таблицу со списком фильмов и уровнем

In [109]:
# открыли таблицу со списком фильмов и уровнями 
df_tab = pd.read_excel(r'C:\Users\nata-\Documents\Masterskaya\movies_labels.xlsx')

In [110]:
df_tab

,id,Movie,Level
0,0,10_Cloverfield_lane(2016),B1
1,1,10_things_I_hate_about_you(1999),B1
2,2,A_knights_tale(2001),B2
3,3,A_star_is_born(2018),B2
4,4,Aladdin(1992),A2/A2+
5,5,All_dogs_go_to_heaven(1989),A2/A2+
6,6,An_American_tail(1986),A2/A2+
7,7,Babe(1995),A2/A2+
8,8,Back_to_the_future(1985),A2/A2+
9,9,Banking_On_Bitcoin(2016),C1


Откроем папку с субтирами в Subtitles_all \ Subtitles

In [111]:
# собрали субтитры из общей папки с субтитрами
basepath_all = r'C:\Users\nata-\Documents\Masterskaya\Subtitles_all\Subtitles'

names = []
subs = []
targets = []
 

for file in os.listdir(basepath_all): #проходим по списку имен файлов в каталоге
    infile = pysrt.open(os.path.join(basepath_all, file), encoding='iso-8859-1')
    base=os.path.basename(file) # возвращает базовое имя пути
    name = os.path.splitext(base)[0] # Разделить путь на имя файла и его расширение
    file_ext = os.path.splitext(base)[1]
    # Если файл не содержит субтитры, пропускаем его
    if file_ext != '.srt':
        continue
 
    txt = infile.text #читаем весь файл
 
    # Сохраняем данные в списки
    names.append(name)
    subs.append(txt)

df = pd.DataFrame({'Movie': names, 'Sub_all': subs}) # собираем датафрейм

In [112]:
pd.set_option('display.max_rows', None)
df

,Movie,Sub_all
0,10_Cloverfield_lane(2016),"<font color=""#ffff80""><b>Fixed & Synced by boz..."
1,10_things_I_hate_about_you(1999),"Hey!\nI'll be right with you.\nSo, Cameron. He..."
2,Aladdin(1992),"<i>Oh, I come from a land\nFrom a faraway plac..."
3,All_dogs_go_to_heaven(1989),CAPTIONING MADE POSSIBLE BY\nMGM HOME ENTERTAI...
4,An_American_tail(1986),(INDISTINCT CONVERSATION)\n(ALL LAUGHING)\nMAM...
5,A_knights_tale(2001),Resync: Xenzai[NEF]\nRETAIL\nShould we help hi...
6,A_star_is_born(2018),"- <i><font color=""#ffffff""> Synced and correct..."
7,Babe(1995),This is a tale about<br/>an unprejudiced heart...
8,Back_to_the_future(1985),"October is inventory time,\nso right now, Stat..."
9,Banking_On_Bitcoin(2016),Downloaded from\nYTS.MX\nOfficial YIFY movies ...


In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Movie    115 non-null    object
 1   Sub_all  115 non-null    object
dtypes: object(2)
memory usage: 1.9+ KB


Откроем папку с субтитрами, которая была отдельно

In [114]:
# собрали субтитры из общей папки с субтитрами
basepath_86 = r'C:\Users\nata-\Documents\Masterskaya\Subtitles'

names = []
subs = []
targets = []
 

for file in os.listdir(basepath_86): #проходим по списку имен файлов в каталоге
    infile = pysrt.open(os.path.join(basepath_86, file), encoding='iso-8859-1')
    base=os.path.basename(file) # возвращает базовое имя пути
    name = os.path.splitext(base)[0] # Разделить путь на имя файла и его расширение
    file_ext = os.path.splitext(base)[1]
    # Если файл не содержит субтитры, пропускаем его
    if file_ext != '.srt':
        continue
 
    txt = infile.text #читаем весь файл
 
    # Сохраняем данные в списки
    names.append(name)
    subs.append(txt)

df_86 = pd.DataFrame({'Movie': names, 'Sub_all': subs}) # собираем датафрейм

In [115]:
df_86

,Movie,Sub_all
0,10_Cloverfield_lane(2016),"<font color=""#ffff80""><b>Fixed & Synced by boz..."
1,10_things_I_hate_about_you(1999),"Hey!\nI'll be right with you.\nSo, Cameron. He..."
2,Aladdin(1992),"<i>Oh, I come from a land\nFrom a faraway plac..."
3,All_dogs_go_to_heaven(1989),CAPTIONING MADE POSSIBLE BY\nMGM HOME ENTERTAI...
4,An_American_tail(1986),(INDISTINCT CONVERSATION)\n(ALL LAUGHING)\nMAM...
5,A_knights_tale(2001),Resync: Xenzai[NEF]\nRETAIL\nShould we help hi...
6,A_star_is_born(2018),"- <i><font color=""#ffffff""> Synced and correct..."
7,Babe(1995),This is a tale about<br/>an unprejudiced heart...
8,Back_to_the_future(1985),"October is inventory time,\nso right now, Stat..."
9,Batman_begins(2005),"Rachel, let me see.\n- Can I see?\n- Finders k..."


Соберем файлы в один датафрейм

In [116]:
data_total = df_tab.merge(df, on='Movie', how='outer')

In [117]:
data_total

,id,Movie,Level,Sub_all
0,0.0,10_Cloverfield_lane(2016),B1,"<font color=""#ffff80""><b>Fixed & Synced by boz..."
1,1.0,10_things_I_hate_about_you(1999),B1,"Hey!\nI'll be right with you.\nSo, Cameron. He..."
2,2.0,A_knights_tale(2001),B2,Resync: Xenzai[NEF]\nRETAIL\nShould we help hi...
3,3.0,A_star_is_born(2018),B2,"- <i><font color=""#ffffff""> Synced and correct..."
4,4.0,Aladdin(1992),A2/A2+,"<i>Oh, I come from a land\nFrom a faraway plac..."
5,5.0,All_dogs_go_to_heaven(1989),A2/A2+,CAPTIONING MADE POSSIBLE BY\nMGM HOME ENTERTAI...
6,6.0,An_American_tail(1986),A2/A2+,(INDISTINCT CONVERSATION)\n(ALL LAUGHING)\nMAM...
7,7.0,Babe(1995),A2/A2+,This is a tale about<br/>an unprejudiced heart...
8,8.0,Back_to_the_future(1985),A2/A2+,"October is inventory time,\nso right now, Stat..."
9,9.0,Banking_On_Bitcoin(2016),C1,Downloaded from\nYTS.MX\nOfficial YIFY movies ...


In [118]:
data_total.rename(columns = {'Sub_all': 'Sub'}, inplace = True) # Переименуем столбцеы 'Sub_all'
data_total = data_total.drop(columns=['id']) # удалим стобец 'id' (249 инд.)

In [119]:
data_total = data_total.merge(df_86, on='Movie', how='outer')

In [120]:
data_total

,Movie,Level,Sub,Sub_all
0,10_Cloverfield_lane(2016),B1,"<font color=""#ffff80""><b>Fixed & Synced by boz...","<font color=""#ffff80""><b>Fixed & Synced by boz..."
1,10_things_I_hate_about_you(1999),B1,"Hey!\nI'll be right with you.\nSo, Cameron. He...","Hey!\nI'll be right with you.\nSo, Cameron. He..."
2,A_knights_tale(2001),B2,Resync: Xenzai[NEF]\nRETAIL\nShould we help hi...,Resync: Xenzai[NEF]\nRETAIL\nShould we help hi...
3,A_star_is_born(2018),B2,"- <i><font color=""#ffffff""> Synced and correct...","- <i><font color=""#ffffff""> Synced and correct..."
4,Aladdin(1992),A2/A2+,"<i>Oh, I come from a land\nFrom a faraway plac...","<i>Oh, I come from a land\nFrom a faraway plac..."
5,All_dogs_go_to_heaven(1989),A2/A2+,CAPTIONING MADE POSSIBLE BY\nMGM HOME ENTERTAI...,CAPTIONING MADE POSSIBLE BY\nMGM HOME ENTERTAI...
6,An_American_tail(1986),A2/A2+,(INDISTINCT CONVERSATION)\n(ALL LAUGHING)\nMAM...,(INDISTINCT CONVERSATION)\n(ALL LAUGHING)\nMAM...
7,Babe(1995),A2/A2+,This is a tale about<br/>an unprejudiced heart...,This is a tale about<br/>an unprejudiced heart...
8,Back_to_the_future(1985),A2/A2+,"October is inventory time,\nso right now, Stat...","October is inventory time,\nso right now, Stat..."
9,Banking_On_Bitcoin(2016),C1,Downloaded from\nYTS.MX\nOfficial YIFY movies ...,NaN


In [121]:
# заменим NaN в столбце Sub на субтитры в столбце Sub_all
data_total['Sub'] = data_total['Sub'].fillna(data_total['Sub_all']) 

In [122]:
data_total

,Movie,Level,Sub,Sub_all
0,10_Cloverfield_lane(2016),B1,"<font color=""#ffff80""><b>Fixed & Synced by boz...","<font color=""#ffff80""><b>Fixed & Synced by boz..."
1,10_things_I_hate_about_you(1999),B1,"Hey!\nI'll be right with you.\nSo, Cameron. He...","Hey!\nI'll be right with you.\nSo, Cameron. He..."
2,A_knights_tale(2001),B2,Resync: Xenzai[NEF]\nRETAIL\nShould we help hi...,Resync: Xenzai[NEF]\nRETAIL\nShould we help hi...
3,A_star_is_born(2018),B2,"- <i><font color=""#ffffff""> Synced and correct...","- <i><font color=""#ffffff""> Synced and correct..."
4,Aladdin(1992),A2/A2+,"<i>Oh, I come from a land\nFrom a faraway plac...","<i>Oh, I come from a land\nFrom a faraway plac..."
5,All_dogs_go_to_heaven(1989),A2/A2+,CAPTIONING MADE POSSIBLE BY\nMGM HOME ENTERTAI...,CAPTIONING MADE POSSIBLE BY\nMGM HOME ENTERTAI...
6,An_American_tail(1986),A2/A2+,(INDISTINCT CONVERSATION)\n(ALL LAUGHING)\nMAM...,(INDISTINCT CONVERSATION)\n(ALL LAUGHING)\nMAM...
7,Babe(1995),A2/A2+,This is a tale about<br/>an unprejudiced heart...,This is a tale about<br/>an unprejudiced heart...
8,Back_to_the_future(1985),A2/A2+,"October is inventory time,\nso right now, Stat...","October is inventory time,\nso right now, Stat..."
9,Banking_On_Bitcoin(2016),C1,Downloaded from\nYTS.MX\nOfficial YIFY movies ...,NaN


In [123]:
data_total = data_total.drop(columns=['Sub_all']) # удалим стобец 'Sub_all' он больше не нужен

In [124]:
data_total = data_total.merge(data, on='Movie', how='outer')

In [125]:
data_total

,Movie,Level_x,Sub_x,Sub_y,Level_y
0,10_Cloverfield_lane(2016),B1,"<font color=""#ffff80""><b>Fixed & Synced by boz...",NaN,NaN
1,10_things_I_hate_about_you(1999),B1,"Hey!\nI'll be right with you.\nSo, Cameron. He...",NaN,NaN
2,A_knights_tale(2001),B2,Resync: Xenzai[NEF]\nRETAIL\nShould we help hi...,NaN,NaN
3,A_star_is_born(2018),B2,"- <i><font color=""#ffffff""> Synced and correct...",NaN,NaN
4,Aladdin(1992),A2/A2+,"<i>Oh, I come from a land\nFrom a faraway plac...",NaN,NaN
5,All_dogs_go_to_heaven(1989),A2/A2+,CAPTIONING MADE POSSIBLE BY\nMGM HOME ENTERTAI...,NaN,NaN
6,An_American_tail(1986),A2/A2+,(INDISTINCT CONVERSATION)\n(ALL LAUGHING)\nMAM...,NaN,NaN
7,Babe(1995),A2/A2+,This is a tale about<br/>an unprejudiced heart...,NaN,NaN
8,Back_to_the_future(1985),A2/A2+,"October is inventory time,\nso right now, Stat...",NaN,NaN
9,Banking_On_Bitcoin(2016),C1,Downloaded from\nYTS.MX\nOfficial YIFY movies ...,NaN,NaN


In [126]:
data_total.isna().sum()

Movie        0
Level_x     50
Sub_x      171
Sub_y      128
Level_y    128
dtype: int64

In [127]:
# заменим NaN в столбце Sub_x на субтитры в столбце Sub_y
data_total['Sub_x'] = data_total['Sub_x'].fillna(data_total['Sub_y'])

# заменим NaN в столбце Level_x на субтитры в столбце Level_y
data_total['Level_x'] = data_total['Level_x'].fillna(data_total['Level_y'])

In [128]:
data_total

,Movie,Level_x,Sub_x,Sub_y,Level_y
0,10_Cloverfield_lane(2016),B1,"<font color=""#ffff80""><b>Fixed & Synced by boz...",NaN,NaN
1,10_things_I_hate_about_you(1999),B1,"Hey!\nI'll be right with you.\nSo, Cameron. He...",NaN,NaN
2,A_knights_tale(2001),B2,Resync: Xenzai[NEF]\nRETAIL\nShould we help hi...,NaN,NaN
3,A_star_is_born(2018),B2,"- <i><font color=""#ffffff""> Synced and correct...",NaN,NaN
4,Aladdin(1992),A2/A2+,"<i>Oh, I come from a land\nFrom a faraway plac...",NaN,NaN
5,All_dogs_go_to_heaven(1989),A2/A2+,CAPTIONING MADE POSSIBLE BY\nMGM HOME ENTERTAI...,NaN,NaN
6,An_American_tail(1986),A2/A2+,(INDISTINCT CONVERSATION)\n(ALL LAUGHING)\nMAM...,NaN,NaN
7,Babe(1995),A2/A2+,This is a tale about<br/>an unprejudiced heart...,NaN,NaN
8,Back_to_the_future(1985),A2/A2+,"October is inventory time,\nso right now, Stat...",NaN,NaN
9,Banking_On_Bitcoin(2016),C1,Downloaded from\nYTS.MX\nOfficial YIFY movies ...,NaN,NaN


In [129]:
data_total = data_total.drop(columns=['Sub_y', 'Level_y']) # удалим лишние стобцы
data_total.rename(columns = {'Sub_x': 'Sub', 'Level_x':'Level'}, inplace = True) # переименуем столбцы

In [130]:
data_total

,Movie,Level,Sub
0,10_Cloverfield_lane(2016),B1,"<font color=""#ffff80""><b>Fixed & Synced by boz..."
1,10_things_I_hate_about_you(1999),B1,"Hey!\nI'll be right with you.\nSo, Cameron. He..."
2,A_knights_tale(2001),B2,Resync: Xenzai[NEF]\nRETAIL\nShould we help hi...
3,A_star_is_born(2018),B2,"- <i><font color=""#ffffff""> Synced and correct..."
4,Aladdin(1992),A2/A2+,"<i>Oh, I come from a land\nFrom a faraway plac..."
5,All_dogs_go_to_heaven(1989),A2/A2+,CAPTIONING MADE POSSIBLE BY\nMGM HOME ENTERTAI...
6,An_American_tail(1986),A2/A2+,(INDISTINCT CONVERSATION)\n(ALL LAUGHING)\nMAM...
7,Babe(1995),A2/A2+,This is a tale about<br/>an unprejudiced heart...
8,Back_to_the_future(1985),A2/A2+,"October is inventory time,\nso right now, Stat..."
9,Banking_On_Bitcoin(2016),C1,Downloaded from\nYTS.MX\nOfficial YIFY movies ...


In [131]:
data_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 291 entries, 0 to 290
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Movie   291 non-null    object
 1   Level   281 non-null    object
 2   Sub     283 non-null    object
dtypes: object(3)
memory usage: 9.1+ KB


**Результат:** итоговая таблица содержит 291 строку.

### Обработка данных

#### Проверим явные дубликаты

In [132]:
data_total.duplicated().sum()

2

In [133]:
duplicate = data_total[data_total.duplicated()] #посмотрим на эти строки
display(duplicate)

,Movie,Level,Sub
39,Powder(1995),B1,"EMT 1:\nAll right, people,\nlet's move! Let's ..."
45,Inside_out(2015),B1,Advertise your product or brand here<br/>conta...


In [134]:
data_total = data_total.drop_duplicates() #удалим явные дубликаты 
data_total.reset_index(drop= True , inplace= True )

#### Проверим неявные дубликаты

In [136]:
movie_list=data_total['Movie'].to_list() # посмотрим список фильмов
movie_list

['10_Cloverfield_lane(2016)',
 '10_things_I_hate_about_you(1999)',
 'A_knights_tale(2001)',
 'A_star_is_born(2018)',
 'Aladdin(1992)',
 'All_dogs_go_to_heaven(1989)',
 'An_American_tail(1986)',
 'Babe(1995)',
 'Back_to_the_future(1985)',
 'Banking_On_Bitcoin(2016)',
 'Batman_begins(2005)',
 'Beauty_and_the_beast(2017)',
 'Before_I_go_to_sleep(2014)',
 'Before_sunrise(1995)',
 'Before_sunset(2004)',
 'Braveheart(1995)',
 'Bridget_Jones_diary(2001)',
 'Bridget_Joness_Baby',
 'Cars(2006)',
 'Cast_away(2000)',
 'Catch_me_if_you_can(2002)',
 'Charlie_and_the_Chocolate_Factory',
 'Cinderella(1950)',
 'Clueless(1995)',
 'Deadpool(2016)',
 'Despicable_Me(2010)',
 'Die_hard(1988)',
 'Dredd(2012)',
 'Dune(2021)',
 'Enola_Holmes(2020)',
 'Entrapment',
 'Eurovision_song_contest_(2020)',
 'Ferdinand(2017)',
 'Fight_club(1999)',
 'Finding_Nemo(2003)',
 'Forrest_Gump(1994)',
 'Good_Will_Hunting(1997)',
 'Groundhog_day(1993)',
 'Powder(1995)',
 'Her(2013)',
 'Home_alone(1990)',
 'Hook(1991)',
 'House_

 Найдено несколько неявных дубликатов, которые надо обработать

'Its_a_wonderful_life(1946)',
'It_s_a_wonderful_life(1946)', 248

'The_blind_side(2009)',
'The_blind_side(2009)',234

'The_terminal(2004)',
'The_terminal(2004)', 257

'Up(2009)', перенести уровень
'Up (2009)', 284

In [137]:
mask = data_total["Movie"].str.contains(r"wonderful")
res = data_total.loc[mask]
display(res)

,Movie,Level,Sub
44,Its_a_wonderful_life(1946),A2/A2+,"I owe everything to George Bailey.\nHelp him, ..."
248,It_s_a_wonderful_life(1946),NaN,"I owe everything to George Bailey.\nHelp him, ..."


In [138]:
mask = data_total["Movie"].str.contains(r"blind_side")
res = data_total.loc[mask]
display(res)

,Movie,Level,Sub
73,The_blind_side(2009),B2,[CROWD CHEERING IN DISTANCE]\nLElGH ANNE: Ther...
74,The_blind_side(2009),B1,[CROWD CHEERING IN DISTANCE]\nLElGH ANNE: Ther...


In [139]:
mask = data_total["Movie"].str.contains(r"The_terminal")
res = data_total.loc[mask]
display(res)

,Movie,Level,Sub
82,The_terminal(2004),B1,"<font color=""#D900D9"">(AIRPLANE APPROACHES)</f..."
83,The_terminal(2004),"A2/A2+, B1","<font color=""#D900D9"">(AIRPLANE APPROACHES)</f..."


In [140]:
mask = data_total["Movie"].str.contains(r"Up")
res = data_total.loc[mask]
display(res)

,Movie,Level,Sub
104,Up (2009),A2/A2+,NaN
246,Up(2009),NaN,{\i1}Movietown{\i0}\n{\i1}News presents Spotli...


In [141]:
# перенесем уровень в строку, где есть субтитры
data_total.loc[data_total['Movie'] == 'Up(2009)', 'Level'] = 'A2' 

C:\Users\nata-\AppData\Local\Temp\ipykernel_9388\498738506.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_total.loc[data_total['Movie'] == 'Up(2009)', 'Level'] = 'A2' #перенесем уровень, из дубликата в основную строку


Таким образом, надо удалить строки с индексами: 248, 74, 83, 104

In [142]:
data_total = data_total.drop(index=[248, 74, 83, 104]) # удалим неявные дубликаты по индексам
data_total.reset_index(drop= True , inplace= True )
data_total

,Movie,Level,Sub
0,10_Cloverfield_lane(2016),B1,"<font color=""#ffff80""><b>Fixed & Synced by boz..."
1,10_things_I_hate_about_you(1999),B1,"Hey!\nI'll be right with you.\nSo, Cameron. He..."
2,A_knights_tale(2001),B2,Resync: Xenzai[NEF]\nRETAIL\nShould we help hi...
3,A_star_is_born(2018),B2,"- <i><font color=""#ffffff""> Synced and correct..."
4,Aladdin(1992),A2/A2+,"<i>Oh, I come from a land\nFrom a faraway plac..."
5,All_dogs_go_to_heaven(1989),A2/A2+,CAPTIONING MADE POSSIBLE BY\nMGM HOME ENTERTAI...
6,An_American_tail(1986),A2/A2+,(INDISTINCT CONVERSATION)\n(ALL LAUGHING)\nMAM...
7,Babe(1995),A2/A2+,This is a tale about<br/>an unprejudiced heart...
8,Back_to_the_future(1985),A2/A2+,"October is inventory time,\nso right now, Stat..."
9,Banking_On_Bitcoin(2016),C1,Downloaded from\nYTS.MX\nOfficial YIFY movies ...


In [143]:
data_total.isna().sum() # проверим количество пропусков

Movie    0
Level    8
Sub      7
dtype: int64

In [144]:
data_total = data_total.dropna(subset=['Sub']) #удалим пропуски в столбце Субтриты, так как в этом случае 
                                               #у нас нет данных для анализа
data_total = data_total.dropna(subset=['Level']) #удалим пропуски в столбце Уровень
data_total.reset_index(drop= True, inplace= True)
data_total.shape[0]

270

In [145]:
data_total.isna().sum()

Movie    0
Level    0
Sub      0
dtype: int64

#### Проверим явные дубликаты еще раз

In [146]:
data_total.duplicated().sum()

0

#### Проверим уникальные значения и пропуски

In [147]:
data_total.nunique() 

Movie    270
Level      7
Sub      255
dtype: int64

В столбце Субтитры уникальных значений меньше, чем в столбце с Фильмами

In [148]:
data_total['Sub'].duplicated().sum()

15

In [150]:
ids = data_total['Sub'] #посмотрим на эти строки (польностью все дубликаты, включая первый)
data_total[ids.isin(ids[ids.duplicated()])]

,Movie,Level,Sub
203,Suits S04E01 EngSub,C1,
204,Suits S04E02 EngSub,C1,
205,Suits S04E03 EngSub,C1,
206,Suits S04E04 EngSub,C1,
207,Suits S04E05 EngSub,C1,
208,Suits S04E06 EngSub,C1,
209,Suits S04E07 EngSub,C1,
210,Suits S04E08 EngSub,C1,
211,Suits S04E09 EngSub,C1,
212,Suits S04E10 EngSub,C1,


In [152]:
data_total.iloc [203]['Sub'] # В нескольких ячейчках загрузились только кавычки вместо субтитров

''

In [153]:
sub_1 = pysrt.open(r'C:\Users\nata-\Documents\Masterskaya\Subtitles_all\C1\Suits S04E01 EngSub.srt')
data_total.loc[203,'Sub']=sub_1

In [154]:
data_total.query('Movie == "Suits S04E01 EngSub"')

,Movie,Level,Sub
203,Suits S04E01 EngSub,C1,"[1\n00:00:00,969 --> 00:00:02,284\nPreviously ..."


In [155]:
sub_2 = pysrt.open(r'C:\Users\nata-\Documents\Masterskaya\Subtitles_all\C1\Suits S04E02 EngSub.srt')
sub_3 = pysrt.open(r'C:\Users\nata-\Documents\Masterskaya\Subtitles_all\C1\Suits S04E03 EngSub.srt')
sub_4 = pysrt.open(r'C:\Users\nata-\Documents\Masterskaya\Subtitles_all\C1\Suits S04E04 EngSub.srt')
sub_5 = pysrt.open(r'C:\Users\nata-\Documents\Masterskaya\Subtitles_all\C1\Suits S04E05 EngSub.srt')
sub_6 = pysrt.open(r'C:\Users\nata-\Documents\Masterskaya\Subtitles_all\C1\Suits S04E06 EngSub.srt')
sub_7 = pysrt.open(r'C:\Users\nata-\Documents\Masterskaya\Subtitles_all\C1\Suits S04E07 EngSub.srt')
sub_8 = pysrt.open(r'C:\Users\nata-\Documents\Masterskaya\Subtitles_all\C1\Suits S04E08 EngSub.srt')
sub_9 = pysrt.open(r'C:\Users\nata-\Documents\Masterskaya\Subtitles_all\C1\Suits S04E09 EngSub.srt')
sub_10 = pysrt.open(r'C:\Users\nata-\Documents\Masterskaya\Subtitles_all\C1\Suits S04E10 EngSub.srt')
sub_11 = pysrt.open(r'C:\Users\nata-\Documents\Masterskaya\Subtitles_all\C1\Suits S04E11 EngSub.srt')
sub_12 = pysrt.open(r'C:\Users\nata-\Documents\Masterskaya\Subtitles_all\C1\Suits S04E12 EngSub.srt')
sub_13 = pysrt.open(r'C:\Users\nata-\Documents\Masterskaya\Subtitles_all\C1\Suits S04E13 EngSub.srt')
sub_14 = pysrt.open(r'C:\Users\nata-\Documents\Masterskaya\Subtitles_all\C1\Suits S04E14 EngSub.srt')
sub_15 = pysrt.open(r'C:\Users\nata-\Documents\Masterskaya\Subtitles_all\C1\Suits S04E15 EngSub.srt')
sub_16 = pysrt.open(r'C:\Users\nata-\Documents\Masterskaya\Subtitles_all\C1\Suits S04E16 EngSub.srt')

In [156]:
data_total.loc[204,'Sub']=sub_2
data_total.loc[205,'Sub']=sub_3
data_total.loc[206,'Sub']=sub_4
data_total.loc[207,'Sub']=sub_5
data_total.loc[208,'Sub']=sub_6
data_total.loc[209,'Sub']=sub_7
data_total.loc[210,'Sub']=sub_8
data_total.loc[211,'Sub']=sub_9
data_total.loc[212,'Sub']=sub_10
data_total.loc[213,'Sub']=sub_11
data_total.loc[214,'Sub']=sub_12
data_total.loc[215,'Sub']=sub_13
data_total.loc[216,'Sub']=sub_14
data_total.loc[217,'Sub']=sub_15
data_total.loc[218,'Sub']=sub_16

In [157]:
mask = data_total["Movie"].str.contains(r"Suits S04")
res = data_total.loc[mask]
display(res)

,Movie,Level,Sub
203,Suits S04E01 EngSub,C1,"[1\n00:00:00,969 --> 00:00:02,284\nPreviously ..."
204,Suits S04E02 EngSub,C1,"[1\n00:00:00,984 --> 00:00:02,262\nPreviously ..."
205,Suits S04E03 EngSub,C1,"[1\n00:00:00,909 --> 00:00:02,781\nPreviously ..."
206,Suits S04E04 EngSub,C1,"[1\n00:00:00,060 --> 00:00:01,239\nPreviously ..."
207,Suits S04E05 EngSub,C1,"[1\n00:00:03,000 --> 00:00:04,778\nIt's not Wo..."
208,Suits S04E06 EngSub,C1,"[1\n00:00:01,034 --> 00:00:02,227\nPreviously ..."
209,Suits S04E07 EngSub,C1,"[1\n00:00:01,000 --> 00:00:02,340\nPreviously ..."
210,Suits S04E08 EngSub,C1,"[1\n00:00:00,806 --> 00:00:02,516\nPreviously ..."
211,Suits S04E09 EngSub,C1,"[1\n00:00:00,146 --> 00:00:01,333\nPreviously ..."
212,Suits S04E10 EngSub,C1,"[1\n00:00:01,705 --> 00:00:02,910\nPreviously ..."


Проверим уникальные значения в столбце Уровень

In [158]:
for value in [data_total]: # выведем значения на экран
    for column in ['Level']:        
        print('\033[1m' + f'Количество уникальных значений {column}:'+ '\033[0m', value[column].nunique())
        print(value[column].unique())
        print()

Количество уникальных значений Level: 7
['B1' 'B2' 'A2/A2+' 'C1' 'B1, B2' 'A2/A2+, B1' 'A2']



In [159]:
# заменим неоднозначные значения на более высокий уровень
data_total['Level'] = data_total['Level']. replace(['A2/A2+', 'B1, B2', 'A2/A2+, B1'], ['A2', 'B2', 'B1'])
data_total['Level'].unique()

array(['B1', 'B2', 'A2', 'C1'], dtype=object)

In [160]:
data_total # проверим полученную таблицу

,Movie,Level,Sub
0,10_Cloverfield_lane(2016),B1,"<font color=""#ffff80""><b>Fixed & Synced by boz..."
1,10_things_I_hate_about_you(1999),B1,"Hey!\nI'll be right with you.\nSo, Cameron. He..."
2,A_knights_tale(2001),B2,Resync: Xenzai[NEF]\nRETAIL\nShould we help hi...
3,A_star_is_born(2018),B2,"- <i><font color=""#ffffff""> Synced and correct..."
4,Aladdin(1992),A2,"<i>Oh, I come from a land\nFrom a faraway plac..."
5,All_dogs_go_to_heaven(1989),A2,CAPTIONING MADE POSSIBLE BY\nMGM HOME ENTERTAI...
6,An_American_tail(1986),A2,(INDISTINCT CONVERSATION)\n(ALL LAUGHING)\nMAM...
7,Babe(1995),A2,This is a tale about<br/>an unprejudiced heart...
8,Back_to_the_future(1985),A2,"October is inventory time,\nso right now, Stat..."
9,Banking_On_Bitcoin(2016),C1,Downloaded from\nYTS.MX\nOfficial YIFY movies ...


In [161]:
data_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Movie   270 non-null    object
 1   Level   270 non-null    object
 2   Sub     270 non-null    object
dtypes: object(3)
memory usage: 6.5+ KB


**Выводы:** в разделе загружены таблицы исходных данных и собраны в один Датафрейм, из таблицы удалены и обработаны все пропуски и дубликаты. Итоговая таблица содержит 270 строк

### Обработка субтитров

Очищаем текст от всего, кроме слов

In [162]:
del_n = re.compile('\n')                # перенос каретки
del_tags = re.compile('<[^>]*>')        # html-теги
del_brackets = re.compile('\([^)]*\)')  # содержимое круглых скобок
clean_text = re.compile('[^а-яa-z\s]')  # все небуквенные символы кроме пробелов
del_spaces = re.compile('\s{2,}')

def prepare_text(text):
    text = del_n.sub(' ', str(text).lower())
    text = del_tags.sub('', text)
    text = del_brackets.sub('', text)
    res_text = clean_text.sub('', text)
    return del_spaces.sub(' ',res_text)

In [163]:
data_total['Sub'] = data_total['Sub'].apply(prepare_text)
#data_total.iloc [0]['Sub'] # проверим результат

Убираем стоп-слова

In [164]:
stop_words = stopwords.words('english')
#print(stop_words)

In [165]:
def del_stopwords(text):
    clean_tokens = tuple(
        map( lambda x: x if x not in stop_words else '', word_tokenize(text) )
    )
    res_text = ' '.join(clean_tokens)
    return res_text

In [166]:
data_total['Sub'] = data_total['Sub'].apply(del_stopwords)
#data_total.iloc [0]['Sub']

In [167]:
type(data_total.iloc [0]['Sub'])

str

Лемматизация

In [168]:
nlp = spacy.load("en_core_web_sm")

In [170]:
def lemmatize(text):    
    doc = nlp(text)
    lemmatized_text = ' '.join([token.lemma_ for token in doc])
    #res_text_lem = ' '.join(clean_tokens)
    return lemmatized_text

In [171]:
data_total['Sub'] = data_total['Sub'].apply(lemmatize)
#data_total.iloc [0]['Sub']

In [172]:
type(data_total.iloc [0]['Sub'])

str

**Результат:**

Субтиры обработаны: из них удалены все знаки кроме буквенных, удалены стоп-слова и проведена лемматизация с помощью SpaCy.

## Обучение моделей для классификации документов

### Подготовка данных для обучения

In [173]:
X = data_total['Sub'].values
y = data_total['Level'].values

In [174]:
# разделим на выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) 
print(X_train.shape)
print(X_test.shape)

(216,)
(54,)


### LogisticRegression

In [175]:
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

In [176]:
param_grid = [{'vect__ngram_range': [(1, 1)],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]}
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

In [177]:
%%time
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


C:\Users\nata-\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
15 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\nata-\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\nata-\anaconda3\lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "C:\Users\nata-\anaconda3\lib\site-packages\sklearn\pipeline.py", line 420, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\n

CPU times: total: 9.42 s
Wall time: 42.8 s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1)]}],
             scoring='accuracy', verbose=1)

In [178]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

Best parameter set: {'clf__C': 100.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1)} 
CV Accuracy: 0.657


In [179]:
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

Test Accuracy: 0.704


Модель **LogisticRegression** обучилась за 41.7 секунд, Accuracy тренировочной выборки 0.657, Accuracy тестовой выборки 0.704.

### LGBMClassifier

In [180]:
model_lgb = LGBMClassifier()

lgb_tfidf = Pipeline([('vect', tfidf),
                     ('clf', model_lgb)])

param_grid = {
    'clf__num_leaves': [30, 50],
    'clf__reg_alpha': [0.1, 0.5],
    'clf__reg_lambda ': [0.1, 0.5]
    }
gs_lgb = GridSearchCV(lgb_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1)

In [181]:
%%time
model_lgb_gs = gs_lgb.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[LightGBM] [Warning] reg_lambda is set=0.0, reg_lambda= will be ignored. Current value: reg_lambda=0.0
[LightGBM] [Warning] Unknown parameter: 0.1
[LightGBM] [Warning] reg_lambda is set=0.0, reg_lambda= will be ignored. Current value: reg_lambda=0.0
[LightGBM] [Warning] Unknown parameter: 0.1
[LightGBM] [Warning] reg_lambda is set=0.0, reg_lambda= will be ignored. Current value: reg_lambda=0.0
[LightGBM] [Warning] Unknown parameter: 0.1
[LightGBM] [Warning] reg_lambda is set=0.0, reg_lambda= will be ignored. Current value: reg_lambda=0.0
[LightGBM] [Warning] Unknown parameter: 0.1
[LightGBM] [Warning] reg_lambda is set=0.0, reg_lambda= will be ignored. Current value: reg_lambda=0.0
[LightGBM] [Warning] Unknown parameter: 0.1
[LightGBM] [Warning] reg_lambda is set=0.0, reg_lambda= will be ignored. Current value: reg_lambda=0.0
[LightGBM] [Warning] Unknown parameter: 0.5
[LightGBM] [Warning] reg_lambda is set=0.0, reg_lambda= wi

In [182]:
print('Best parameter set: %s ' % model_lgb_gs.best_params_)
print('CV Accuracy: %.3f' % model_lgb_gs.best_score_)

Best parameter set: {'clf__num_leaves': 30, 'clf__reg_alpha': 0.1, 'clf__reg_lambda ': 0.1} 
CV Accuracy: 0.611


In [183]:
clf = model_lgb_gs.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

Test Accuracy: 0.667


Модель **LGBMClassifier** обучилась за 1 мин. 30 сек., Accuracy тренировочной выборки 0.611, Accuracy тестовой выборки 0.667. 



### RandomForestClassifier

In [184]:
model_rf = RandomForestClassifier(class_weight='balanced')
rf_tfidf = Pipeline([('vect', tfidf),
                     ('clf', model_rf)])

In [185]:
%%time
params_rf = {
    'clf__n_estimators': [10, 20],
    'clf__max_depth': [4, 6, 10],
    'clf__random_state': [13]
}

grid_rf = GridSearchCV(rf_tfidf, param_grid=params_rf, cv=5, 
                       scoring='accuracy').fit(X_train, y_train) 

CPU times: total: 33.3 s
Wall time: 34.1 s


In [186]:
print('Best parameter set: %s ' % grid_rf.best_params_)
print('CV Accuracy: %.3f' % grid_rf.best_score_)

Best parameter set: {'clf__max_depth': 10, 'clf__n_estimators': 10, 'clf__random_state': 13} 
CV Accuracy: 0.616


In [187]:
clf = grid_rf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

Test Accuracy: 0.574


Модель **RandomForestClassifier** обучилась за 35.6 сек., Accuracy тренировочной выборки 0.616, Accuracy тестовой выборки 0.574. 

### GradientBoostingClassifier

In [188]:
class_weights = compute_class_weight("balanced",
                                    classes = np.unique(y_train),
                                    y = y_train                                                    
                                    )
class_weights = dict(zip(np.unique(y_train), class_weights))
class_weights

{'A2': 2.0,
 'B1': 1.148936170212766,
 'B2': 0.4778761061946903,
 'C1': 1.8620689655172413}

In [189]:
sample_weights=compute_sample_weight(class_weight='balanced', y=y_train)

In [190]:
model_gb = GradientBoostingClassifier()

gb_tfidf = Pipeline([('vect', tfidf),
                     ('clf', model_gb)])

parameters = [{ 
              'clf__max_depth': [3],
              'clf__learning_rate': [0.2, 0.3],
              'clf__subsample' : [1],
             # 'clf__n_estimators' : [200, 300],
              'clf__loss': ['log_loss'],
              'clf__max_features':[None] 
}]

grid_search_gb = GridSearchCV(
                              gb_tfidf,
                              parameters,
                              cv = 3, 
                              verbose = 1,
                              scoring = 'accuracy'
                            )

In [191]:
%%time
grid_search_gb.fit(X_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
CPU times: total: 14min 52s
Wall time: 15min 17s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf', GradientBoostingClassifier())]),
             param_grid=[{'clf__learning_rate': [0.2, 0.3],
                          'clf__loss': ['log_loss'], 'clf__max_depth': [3],
                          'clf__max_features': [None], 'clf__subsample': [1]}],
             scoring='accuracy', verbose=1)

In [192]:
print(grid_search_gb.best_params_, grid_search_gb.best_score_)

{'clf__learning_rate': 0.2, 'clf__loss': 'log_loss', 'clf__max_depth': 3, 'clf__max_features': None, 'clf__subsample': 1} 0.6296296296296297


In [193]:
clf = grid_search_gb.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

Test Accuracy: 0.722


Модель **GradientBoostingClassifier** обучилась за 15 мин. 17 сек., Accuracy тренировочной выборки 0.629, Accuracy тестовой выборки 0.722.

### Сравнение моделей

In [194]:
df = [['41.7s', 0.657, 0.704],
        ["1min 30s", 0.611, 0.667],
        ["35.6s", 0.616, 0.574],
        ["15min 17s", 0.629, 0.722]]
model = ["LogisticRegression", "LGBMClassifier", "RandomForestClassifier", "GradientBoostingClassifier"]

In [195]:
pd.DataFrame(data=df, index=model, 
             columns=["Время обучения модели", "Accuracy тренировочной выборки", 
                      "Accuracy тестовой выборки"])

,Время обучения модели,Accuracy тренировочной выборки,Accuracy тестовой выборки
LogisticRegression,41.7s,0.657,0.704
LGBMClassifier,1min 30s,0.611,0.667
RandomForestClassifier,35.6s,0.616,0.574
GradientBoostingClassifier,15min 17s,0.629,0.722


## Выводы

В ходе работы загружены исходные данные и собраны в один Датафрейм, из таблицы удалены и обработаны все пропуски и дубликаты. Итоговая таблица содержит 270 строк.

Субтиры обработаны: из них удалены все знаки кроме буквенных, удалены стоп-слова и проведена лемматизация с помощью SpaCy.

Данные разделены на выборки и проведена векторизация с помощью TfidfVectorizer.

Было исслледовано 4 модели: **LogisticRegression, LGBMClassifier, RandomForestClassifier, GradientBoostingClassifier**. 

Результаты исследования:

- Модель **LogisticRegression** обучилась за 41.7 секунд, Accuracy тренировочной выборки 0.657, Accuracy тестовой выборки 0.704.
- Модель **LGBMClassifier** обучилась за 1 мин. 30 сек., Accuracy тренировочной выборки 0.611, Accuracy тестовой выборки 0.667.
- Модель **RandomForestClassifier** обучилась за 35.6 сек., Accuracy тренировочной выборки 0.616, Accuracy тестовой выборки 0.574.
- Модель **GradientBoostingClassifier** обучилась за 15 мин. 17 сек., Accuracy тренировочной выборки 0.629, Accuracy тестовой выборки 0.722.

При исследовании моделей лучше всего себя показала модель **LogisticRegression** на тренировочной и тестовой выборке. Так же время оучения модели оказалось наименьшим. По показателю Accuracy хуже всего себя показала модель **RandomForestClassifier**. По времени обучения хуже всех оказалась модель  **GradientBoostingClassifier**.

In [197]:
# сохранение лучшей модели
best_model_lr = gs_lr_tfidf.best_estimator_
with open(r'C:\Users\nata-\Documents\Masterskaya\model_lr_2.pkl', "wb") as dataset:
    pickle.dump(best_model_lr, dataset)